# TNFR Visualization Examples

This notebook demonstrates deterministic TNFR telemetry visualizations
powered by the helpers in `tnfr.viz.matplotlib`. Execute the cells to
produce coherence, phase synchrony, and spectral figures driven by
reproducible synthetic data.

## Reproducibility contract

The synthetic signals are generated from a fixed NumPy random seed and
all relevant library versions are logged. The resulting figures can be
regenerated on any system with the same dependency set by re-running the
notebook from top to bottom.

In [ ]:
from __future__ import annotations

import json
from importlib import metadata
from importlib.metadata import PackageNotFoundError

import numpy as np

SEED = 202405
rng = np.random.default_rng(SEED)

try:
    tnfr_version = metadata.version("tnfr")
except PackageNotFoundError:  # Fallback to the local package version
    from tnfr import __version__ as tnfr_version

metadata_report = {
    "seed": SEED,
    "numpy": np.__version__,
    "tnfr": tnfr_version,
}

print(json.dumps(metadata_report, indent=2))

In [ ]:
from tnfr.viz import plot_coherence_matrix, plot_phase_sync, plot_spectrum_path

## Coherence matrix

This example crafts a symmetric coherence matrix that highlights a
coupled quartet of nodes. Running the cell renders the matrix using the
canonical `plot_coherence_matrix` helper.

In [ ]:
channels = ["α", "β", "γ", "δ"]
base = rng.uniform(0.45, 0.9, size=(len(channels), len(channels)))
coherence = (base + base.T) / 2
np.fill_diagonal(coherence, 1.0)

fig, ax = plot_coherence_matrix(coherence, channels=channels)
fig.tight_layout()
fig

## Phase synchrony trajectories

The next cell emits deterministic phase trajectories for three nodes
under a shared structural frequency. The `plot_phase_sync` helper tracks
how synchrony evolves across structural cycles.

In [ ]:
structural_frequency = 3.5  # Hz_str
samples = 240
time_axis = np.linspace(0.0, 12.0, samples)

phase_offsets = rng.uniform(-0.4, 0.4, size=3)
phase_noise = rng.normal(scale=0.03, size=(3, samples))
base_phase = np.sin(time_axis / (np.pi / 1.8))
phase_paths = base_phase + phase_offsets[:, None] + phase_noise

fig, ax = plot_phase_sync(
    phase_paths,
    time_axis,
    structural_frequency=structural_frequency,
    node_labels=["node α", "node β", "node γ"],
)
fig.tight_layout()
fig

## Structural spectrum

Finally, the spectrum helper tracks how coherence redistributes across
structural frequencies. The generated deterministic curve simulates a
stable resonance band with light secondary harmonics.

In [ ]:
frequencies = np.linspace(0.5, 8.0, 256)
peak = np.exp(-0.5 * ((frequencies - 3.2) / 0.6) ** 2)
secondary = 0.35 * np.exp(-0.5 * ((frequencies - 5.6) / 0.8) ** 2)
noise_floor = 0.08 + 0.03 * np.cos(frequencies * 1.7)
spectrum = peak + secondary + noise_floor

fig, ax = plot_spectrum_path(
    frequencies,
    spectrum,
    label="ΔNFR resonance distribution",
)
fig.tight_layout()
fig